In [1]:
#!pip install opencv-python

In [2]:
import threading
import cv2 as cv
import numpy as np

face_cascade = cv.CascadeClassifier("haarcascade_frontalface_default.xml")

In [3]:
def inputWorker(ref):
    while True:
        ref[0] = input("Küsimus")
        if ref[0] == "exit":
            break

        if ref[0] == "fedora":
            ref[1] = "fedora"

In [4]:
def overlayFilter(image, x, y, w, h, filter_img):
    # Võtame overlay pildist alpha kanali välja
    alpha = filter_img[:,:,3]
    alpha = cv.merge([alpha,alpha,alpha])

    cropped_image = image[y:y + h, x:x + w]

    # Võtame overlay pildist BGR kanalid välja
    bgr = filter_img[:,:,0:3]

    # Kui alpha on 0, vali taust, kui mitte, vali meie overlay filter
    image[y:y + h, x:x + w] = np.where(alpha==0, cropped_image, bgr)
    return image

In [5]:
def headFilter(image, x, y, w, h, filter_img):
    filter_img = cv.resize(filter_img, (w, h))
    y_offset = y + int(filter_img.shape[0] * 0.2) - filter_img.shape[0]

    # Kui filter läheb ekraanist välja, siis lõikame vastavalt vähemaks
    if y_offset < 0:
        filter_img = filter_img[abs(y_offset):]
        h -= abs(y_offset)
        y_offset = 0

    if 0 <= x and x + w < image.shape[1] and y_offset + h < image.shape[0]:
        return overlayFilter(image, x, y_offset, w, h, filter_img)
    return image

In [6]:
def filterImage(image, x_cord, y_cord, width, height, filter_name):
    head_filters = ["fedora"]
    eye_filters = ["glasses", "sunglasses"]
    filter_img = cv.imread(f"filters/{filter_name}.png", cv.IMREAD_UNCHANGED)

    if filter_name in head_filters:
        image = headFilter(image, x_cord, y_cord, width, height, filter_img)
    #elif filter_name in eye_filters:
    #    image = eyeFilter(image, x_cord, y_cord, width, height, filter_img)

    return image

In [7]:
video = cv.VideoCapture(0)
refs = ["", ""]

threading.Thread(target=inputWorker, args=[refs]).start()

while True:
    result, img = video.read()

    # Flipib pildi ümber, et toimuks arusaadav peegeldus
    img = cv.flip(img, 1)

    # Näotuvastus
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_img)
    for (x, y, w, h) in faces:
        if refs[1] != "":
            img = filterImage(img, x, y, w, h, refs[1])

    cv.imshow("Camera", img)

    k = cv.waitKey(30) & 0xFF
    if k == 27: # ehk ESC nupp
        break

    if refs[0] == "exit":
        break

video.release()
cv.destroyAllWindows()